In [1]:
# INPUT:
# TODO: Build simple ui or just use terminal for inputs
# TODO: Automatically score scripts that have been found in directory

# SCORING:
# TODO: Make sure to score all trials separately (practices can be scored all at once)
# TODO: All proportion correct should be displayed in percentage format
# TODO: Make sure to average response time
# TODO: Display response times per each response later on
# TODO: Response time correct vs incorrect avgs
# TODO: Score distance using city block method (will be in the nitty gritty area)
# TODO: Score avg distance across trials

# OUTPUT:
# TODO: Open csv for each test scored
# TODO: Show preliminary information first
# TODO: Display more in-depth details further along (horizontal)
# TODO: Display avg distance from correct block (incorrect answers) in the 
# TODO: Place all scored files in scored directory
# TODO: Print LOGS stating what is being done

# FINAL:
# TODO: Migrate .ipynb to .py format and finish syncing script
# TODO: Make script executable?!?!
# TODO: Allow for more in-depth customization in args (argparse)

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot
import os
import logging as log

class Util:
    def __init__(self) -> None:
        pass

    def findFile(self, testName=str):

        # Get the current directory
        currentDir = os.getcwd()

        # Search for a txt file in the current directory
        for file in os.listdir(currentDir):
            if file.endswith('.txt'):
                if testName in file:
                    # If a TXT file is found, read it into a Pandas dataframe
                    df = pd.read_csv(os.path.join(currentDir, file), delimiter='\t')
                    print(f'Successfully read file: {file}')
                    return df
        else:
            # If no txt file is found, print an error message
            print(f'No TXT file named {testName} found in current directory')

    def getTrialType(self, df=pd.DataFrame):
        # List different types of trials in the dataframe
        trialTypes = df.iloc[:, 4].unique()
        return trialTypes
    
    def subsetByTrial(self, df=pd.DataFrame, trialTypes=np.ndarray):
        # Create a list to hold the subsetted dataframes
        subsetList = []
        # Subset data based on trial number
        for trial in trialTypes:
            # Create a new dataframe for each trial type
            trialDf = df[df.iloc[:, 4] == trial]
            # Add each dataframe to a list
            subsetList.append(trialDf)
        # Return the list of dataframes
        return subsetList
    
    
class InputHandler(Util):
    def __init__(self) -> None:
        super().__init__()


class OutputHandler(Util):
    def __init__(self) -> None:
        super().__init__()

myUtil = Util()
myOutputHandler = OutputHandler()


In [33]:
def fsScore():

    # grab test document
    fs = myUtil.findFile('FiguralSpeed')
    
    # determine all unique trials
    trials = myUtil.getTrialType(fs)
    
    # subset data by trial
    trialDfs = myUtil.subsetByTrial(fs, trials)

    # score each trial
    for trial in trialDfs:
        


    # outputs!!!





fsScore()

Successfully read file: 1001_FiguralSpeed.txt
['PRACT  ' 'TRIAL1 ' 'TRIAL2 ']
    1001   2   6   1   PRACT     1   8000   500   2 .1  Fa4_1211   ...  /   \
79   1001   2   6   3  TRIAL2     1   8000   500     1  Fc4_3113   ...  z    
80   1001   2   6   3  TRIAL2     2   8000   500     2  Fc2_3221   ...  /    
81   1001   2   6   3  TRIAL2     3   8000   500     1  Fc2_3332   ...  z    
82   1001   2   6   3  TRIAL2     4   8000   500     1  Fc2_1231   ...  z    
83   1001   2   6   3  TRIAL2     5   8000   500     1  Fc4_2333   ...  z    

   / .1 1 .1  1741   2023   4   5   15   38   21  
79   z     1   2762   2023   4   5   15   42  50  
80   /     1   3038   2023   4   5   15   42  50  
81   z     1   3599   2023   4   5   15   42  50  
82   z     1   3095   2023   4   5   15   42  50  
83   z     1   3718   2023   4   5   15   42  50  

[5 rows x 21 columns]


In [5]:
def luScore():
    pass    

luScore()

<class 'pandas.core.frame.DataFrame'>


In [2]:
def msScore():
    pass

msScore

In [ ]:
def nmScore():
    pass

nmScore()

In [ ]:
# this test has a different trial format and the column is 6 instead of 5
def nbScore():
    
    pass

nbScore()

In [ ]:
def nsScore():
    pass

nsScore()

In [ ]:
def olmScore():
    pass

olmScore()

In [ ]:
def suScore():
    pass

suScore()

In [ ]:
def stScore():
    pass

stScore()

In [ ]:
def vsScore():
    pass

vsScore()

In [ ]:
def wrScore():
    pass

wrScore()